In [1]:
using Pkg
Pkg.activate("/home/wkh/XLA.jl")

"/home/wkh/XLA.jl/Project.toml"

In [2]:
using Distributions
Pkg.resolve()
using Distributions

┌ Info: Recompiling stale cache file /home/wkh/.julia/compiled/v1.1/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1184
┌ Warning: Package Distributions does not have Test in its dependencies:
│ - If you have Distributions checked out for development and have
│   added Test as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Distributions
└ Loading Test into Distributions from project dependency, future warnings for Distributions are suppressed.


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/XLA.jl/Project.toml`
 [no changes]
  Updating `~/XLA.jl/Manifest.toml`
 [no changes]


In [ ]:
using XLA, TensorFlow, Plots

In [ ]:
sess = Session(Graph(); target="grpc://10.240.1.2:8470")
run(sess, TensorFlow.Ops.configure_distributed_tpu())
XLA.initialize!("10.240.1.2:8470"; reset=true);

In [ ]:
f(A) = A*A*A

In [ ]:
hostA = randn(Float32, 10000, 10000)
A = tpu(hostA);

In [ ]:
Pkg.add("BenchmarkTools")
using BenchmarkTools

In [ ]:
@time f(hostA)

In [ ]:
@time f(hostA);

In [ ]:
@benchmark @tpu f(A)

In [ ]:
g() = randn(XRTArray{Float32}, 10000, 10000)
@benchmark @tpu g()

In [ ]:
h() = f(g())
@benchmark @tpu h()

In [ ]:
relu(x) = max(x, zero(x))

In [ ]:
plot(relu, -1.0, 1.0)

In [ ]:
r() = relu.(randn(XRTArray{Float32}, 16, 16))
@tpu r()

In [ ]:
struct Dense{T,S,F}
    W::T
    b::S
    σ::F
end

Base.show(io::IO, d::Dense) = print(io, "Dense", size(d.W))
(d::Dense)(x) = d.σ.(d.W*x + d.b)

In [ ]:
make_one(A) = Dense(A, zeros(XRTArray{Float32, (16,), 1}), relu)

In [ ]:
dense = @tpu make_one(@tpu r())

In [ ]:
df(dense) = dense(randn(XRTArray{Float32}, 16))

In [ ]:
@tpu df(dense)